In [64]:
import pandas as pd 
import sklearn
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn import tree
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import chi2

from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest

from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support


import gensim, logging
from gensim.models import Word2Vec
from scipy import sparse

In [65]:
from nltk.corpus import stopwords

In [66]:
stoplist = stopwords.words('english')

In [67]:
more_stopwords = "based serive services amp /n /r ltd provide year company special  giving established various became 1987 range like every center best quality shop india indian complete range leading concern like time latest every one well-known also south delhi mumbai india indian bangalore hyderabad chennai"
stoplist += more_stopwords.split()

In [68]:
stoplist[0]

u'i'

In [104]:
import pandas as pd 
import sklearn
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn import tree
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import chi2

from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest

from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support


import gensim, logging
from gensim.models import Word2Vec
from scipy import sparse

def loadData(filePath="dataset.csv"):
	data=pd.read_csv("/home/administrator/data/categories-data/Train-Data/fps-with-cat-train.csv") 
	data['CategoryFB'] = data['CategoryFB'].fillna(data['CategoryV2'])
	data['Description'] = data['Description'].fillna(data['Name'])
	return data["Tag"],data["Description"],data["CategoryV2"]

def preProcessing(features):
    num_descs = features.size
    clean_wordlist = []
    clean_descs = []
    stops = set(stopwords.words('english'))
    more_stopwords = "based nwe n r amp ncar nagar nto nand quot k provide service services year giving established various became 1987 range like every center best quality india indian complete range leading concern like time latest every one well-known also south delhi mumbai india indian bangalore hyderabad chennai"
    stops.update(more_stopwords.split())
    for i in range( 0, num_descs):
        letters_only = re.sub("[^a-zA-Z]", " ", features[i]) 
        words = letters_only.lower().split()
        words = [w.lower() for w in words if not w in stops]  
        clean_wordlist.append(words)
        clean_descs.append(" ".join(words))
    return clean_descs, clean_wordlist


def getDTMByTFIDF(features,nfeatures):
    tfIdf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0, stop_words = 'english', max_features=nfeatures)
    dtm = tfIdf_vectorizer.fit_transform(features).toarray()
    return dtm,tfIdf_vectorizer

def featuresByChiSq(features,labels,nFeature=3000):
    chi2_model = SelectKBest(chi2,k=nFeature)
    dtm = chi2_model.fit_transform(features,labels)
    return dtm,chi2_model

In [105]:
def crossValidate(document_term_matrix,labels,classifier="SVM",nfold=10):
    clf = None
    precision = []
    recall = []
    fscore = []
    
    if classifier == "RF":
        clf = RandomForestClassifier(class_weight='auto')
    elif classifier == "NB":
        clf = MultinomialNB()
    elif classifier == "SVM":
        clf = LinearSVC(class_weight='auto')
    
    skf = StratifiedKFold(labels, n_folds=nfold)

    for train_index, test_index in skf:
        X_train, X_test = document_term_matrix[train_index], document_term_matrix[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        model = clf.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        p,r,f,s = precision_recall_fscore_support(y_test, y_pred, average='weighted')
        precision.append(p)
        recall.append(r)
        fscore.append(f)
        
    return round(np.mean(precision),3),round(np.mean(recall),3),round(np.mean(fscore),3)

In [106]:
tags, descs, labels = loadData()
processed_descs, processed_descs_wordlist = preProcessing(descs)

In [107]:
labels[0]

'HARDWARE & SANITARYWARE'

In [108]:
dtm,vect = getDTMByTFIDF(processed_descs,3000)

In [110]:
chisqDtm, chisqModel = featuresByChiSq(dtm,labels,2000)

In [111]:
precision, recall, fscore = crossValidate(chisqDtm,labels,"SVM",10)

/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


In [112]:
print precision, recall, precision_recall_fscore_support

0.93 0.839 <function precision_recall_fscore_support at 0x7f2b751b89b0>


In [113]:
from sklearn.cross_validation import train_test_split

train_descs, test_descs = train_test_split(descs, test_size=0.1, 
                                           random_state=42)

train_labels, test_labels = train_test_split(labels, test_size=0.1, 
                                           random_state=42)

train_descs = train_descs.reset_index(drop=True)
test_descs = test_descs.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

processed__train_descs, processed_train_descs_wordlist = preProcessing(train_descs)
processed_test_descs, processed_test_descs_wordlist = preProcessing(test_descs)

dtm_train,vect_train = getDTMByTFIDF(processed__train_descs,2000)
chisqDtmTrain, chisqModelTrain = featuresByChiSq(dtm_train,train_labels,2000)

clf = LinearSVC(class_weight='auto')

In [118]:
from sklearn.cross_validation import train_test_split

train_descs, test_descs = train_test_split(descs, test_size=0.1, 
                                           random_state=42)

train_labels, test_labels = train_test_split(labels, test_size=0.1, 
                                           random_state=42)

train_descs = train_descs.reset_index(drop=True)
test_descs = test_descs.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_labels = test_labels.reset_index(drop=True)

processed__train_descs, processed_train_descs_wordlist = preProcessing(train_descs)
processed_test_descs, processed_test_descs_wordlist = preProcessing(test_descs)

dtm_train,vect_train = getDTMByTFIDF(processed__train_descs,1500)
chisqDtmTrain, chisqModelTrain = featuresByChiSq(dtm_train,train_labels,1000)

clf = LinearSVC(class_weight='auto', probability = True)

TypeError: __init__() got an unexpected keyword argument 'probability'

In [116]:
model = clf.fit(chisqDtmTrain, train_labels)
dtm_test,vect_test = getDTMByTFIDF(processed_test_descs,1500)
chisqDtmTest, chisqModelTest = featuresByChiSq(dtm_test,test_labels,1000)
y_pred = model.predict(chisqDtmTest)

p,r,f,s = precision_recall_fscore_support(test_labels, y_pred, average='weighted')

/home/administrator/.local/lib/python2.7/site-packages/sklearn/utils/class_weight.py:65: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


In [119]:
print p,r,f,s

0.369468926137 0.290845886443 0.315524252641 None


ValueError: could not convert string to float: EDUCATION

In [54]:
from collections import defaultdict
cdict = defaultdict(list)
for i, val in enumerate(labels):
    if val in cdict:
        cdict[val].extend(processed_descs_wordlist[i])
    else:
        value = list()
        cdict[val] = processed_descs_wordlist[i]

In [55]:
from collections import defaultdict
fq= defaultdict( int )
for w in cdict['HOTEL']:
    fq[w] += 1
    
copy = []
for k,v in fq.items():
    copy.append((v, k))


copy = sorted(copy, reverse=True)

index = 0
for k in copy:
    if index == 50:
        break
    else:
        print '%s: %d' %(k[1], k[0])
    index = index+1

hotel: 3596
hotels: 348
city: 319
rooms: 297
palace: 258
located: 251
restaurant: 224
business: 196
hospitality: 193
luxury: 175
guests: 170
well: 164
facilities: 155
inn: 142
stay: 140
room: 138
international: 136
star: 132
heart: 131
accommodation: 131
residency: 121
budget: 116
comfort: 114
station: 108
mahabaleshwar: 104
comfortable: 102
dreamland: 101
railway: 100
amenities: 98
world: 96
offers: 96
offer: 95
situated: 93
experience: 93
modern: 92
hall: 91
ideal: 90
grand: 88
banquet: 84
luxurious: 82
home: 82
among: 80
place: 79
equipped: 79
bus: 79
class: 78
resort: 77
enjoy: 77
guest: 76
many: 73


In [56]:
from sklearn.metrics import classification_report
print classification_report(test_labels, y_pred)

                         precision    recall  f1-score   support

           ARCHITECTURE       0.00      0.00      0.00         3
                   ARTS       0.00      0.00      0.00         2
             AUTOMOTIVE       0.00      0.00      0.00        17
           BEAUTY & SPA       0.00      0.00      0.00       171
                  BLOGS       0.00      0.00      0.00        13
               CATERING       0.00      0.00      0.00         1
              CHEMICALS       0.00      0.00      0.00         7
           CONSTRUCTION       0.00      0.00      0.00         2
              EDUCATION       0.15      0.09      0.11       400
            ELECTRONICS       0.22      0.42      0.29       396
          ENTERTAINMENT       0.00      0.00      0.00         7
                  EVENT       0.00      0.00      0.00         9
                FASHION       0.02      0.01      0.01       320
                FLORIST       0.00      0.00      0.00         4
       FOOD & BEVERAGES 